# **任务描述：**

### 本次实践是一个多分类任务，需要将照片中的宝石分别进行识别，完成**宝石的识别**

### **实践平台：百度AI实训平台-AI Studio、PaddlePaddle2.0.0 动态图**


![](https://ai-studio-static-online.cdn.bcebos.com/7640d4434f894f5dbae1d85c62e54b8476e24856aabc4c5f9915fea1b26f3ebc)


# 深度神经网络(DNN)

### **深度神经网络（Deep Neural Networks，简称DNN）是深度学习的基础，其结构为input、hidden（可有多层）、output，每层均为全连接。**
![](https://ai-studio-static-online.cdn.bcebos.com/c60fc28848cf469fa3a7824aa637a03f3b2b213ce7b84659919cb24b4430bffb)



# 数据集介绍

* **数据集文件名为archive_train.zip,archive_test.zip。**

* **该数据集包含25个类别不同宝石的图像。**

* **这些类别已经分为训练和测试数据。**

* **图像大小不一，格式为.jpeg。**


![](https://ai-studio-static-online.cdn.bcebos.com/7640d4434f894f5dbae1d85c62e54b8476e24856aabc4c5f9915fea1b26f3ebc)


In [68]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

data55032  dataset


In [69]:
#导入需要的包
import os
import zipfile
import random
import json
import cv2
import numpy as np
from PIL import Image
import paddle
import matplotlib.pyplot as plt
from paddle.io import Dataset
import shutil

# 1、数据准备

In [70]:
'''
参数配置
'''
train_parameters = {
    "input_size": [3, 224, 224],                           #输入图片的shape
    "class_dim": -1,                                     #分类数
    "src_path":"data/data55032/archive_train.zip",       #原始数据集路径
    "target_path":"/home/aistudio/data/dataset",        #要解压的路径 
    "train_list_path": "./train.txt",              #train_data.txt路径
    "eval_list_path": "./eval.txt",                  #eval_data.txt路径
    "label_dict":{},                                    #标签字典
    "readme_path": "/home/aistudio/data/readme.json",   #readme.json路径
    "num_epochs": 100,                                    #训练轮数
    "train_batch_size": 96,                             #批次的大小
    "learning_strategy": {                              #优化函数相关的配置
        "lr": 0.001                                     #超参数学习率
    } 
}



In [71]:
from pathlib import Path
def unzip_data(src_path,target_path):

    '''
    解压原始数据集，将src_path路径下的zip包解压至data/dataset目录下
    '''

    if(not os.path.isdir(target_path)):    
        z = zipfile.ZipFile(src_path, 'r')
        z.extractall(path=target_path)
        z.close()
    else:
        print("文件已解压")
    for _dir in Path(target_path).glob('**/__MACOSX'):
        shutil.rmtree(_dir)
    

In [73]:
'''
参数初始化
'''
src_path=train_parameters['src_path']
target_path=train_parameters['target_path']
train_list_path=train_parameters['train_list_path']
eval_list_path=train_parameters['eval_list_path']
batch_size=train_parameters['train_batch_size']
'''
解压原始数据到指定路径
'''
unzip_data(src_path,target_path)

'''
划分训练集与验证集，乱序，生成数据列表
'''
#每次生成数据列表前，首先清空train.txt和eval.txt

'''   
#生成数据列表   
get_data_list(target_path,train_list_path,eval_list_path)
'''

文件已解压


'   \n#生成数据列表   \nget_data_list(target_path,train_list_path,eval_list_path)\n'

In [74]:
def gen_train_eval():
    from pathlib import Path
    targetPath = Path(target_path)
    class_dirs = sorted(targetPath.glob("*"))
    print(class_dirs)
    train_parameters['class_dim']=len(class_dirs)
    #获取数据 metadata
    lst_data = []
    for i, class_dir in enumerate(class_dirs):
        lst_path = list(class_dir.glob("*.jpg"))
        lst_gemName = [p.parent.name for p in lst_path]
        #zip当前路径、名称以及该路径下的宝石类别,
        #lst_path、lst_genName都是一个list，所以类别也需要是一个list
        lst_data.extend( zip(map(str,lst_path),lst_gemName,[i]*len(lst_path)) )

    import pandas as pd
    #构建数据 dataframe 并且打乱    
    df_data = pd.DataFrame(lst_data,columns=['gem_path','gem_name','lbl']).sample(frac=1,replace=False,random_state=1001)
    df_data.head(5)    

    #得到标签字典，无重复
    dic = df_data[['lbl','gem_name']].drop_duplicates()
    dic.head(5)

# gen_train_eval()

In [75]:
import pandas as pd
if False:
    #构建数据 dataframe 并且打乱    
    df_data = pd.DataFrame(lst_data,columns=['gem_path','gem_name','lbl']).sample(frac=1,replace=False,random_state=1001)
    df_data.head(5)

In [76]:
if False:
    #得到标签字典，无重复
    dic = df_data[['lbl','gem_name']].drop_duplicates()
    dic.head(5)

In [77]:
if False:
    train_parameters['label_dict'] = {v:int(k) for k,v in dic.to_records(index=False)}
    train_parameters

In [78]:
if False:
    from sklearn.model_selection import train_test_split
    train_data, eval_data = train_test_split(df_data, test_size=0.1, random_state=42)

In [86]:
#加载和保存数据
train_df_json='train_data.json'
eval_df_json='eval_data.json'
train_params_path='params.json'
def save_params():
    train_data.to_json(train_df_json)
    eval_data.to_json(eval_df_json)
    with open(train_params_path, 'w') as f:
        tmp_ = json.dumps(train_parameters)
        f.write(tmp_)

def load_params():
    global train_data
    global eval_data
    global train_parameters
    train_data = pd.read_json(train_df_json)
    eval_data = pd.read_json(eval_df_json)
    with open(train_params_path, 'r') as f:
        tmp_str = f.read()
    train_parameters = json.loads(tmp_str)
#save_params()
load_params()

In [87]:
def info( ):
    print("== 数据集总体情况:总类别数", train_data.lbl.max() + 1)
    print("== 训练集不同类别的样本数：")
    print(train_data.gem_name.value_counts())
    print(f"== 训练集样本数：{len(train_data)}", f"验证集样本数：{len(eval_data)}")
info()    

== 数据集总体情况:总类别数 25
== 训练集不同类别的样本数：
Labradorite      34
Beryl Golden     33
Garnet Red       33
Quartz Beer      32
Alexandrite      30
Pearl            30
Iolite           30
Sapphire Blue    29
Carnelian        29
Tanzanite        29
Variscite        29
Fluorite         29
Emerald          29
Danburite        29
Zircon           29
Hessonite        28
Cats Eye         28
Almandine        28
Rhodochrosite    28
Kunzite          28
Jade             28
Benitoite        28
Diamond          27
Malachite        26
Onyx Black       26
Name: gem_name, dtype: int64
== 训练集样本数：729 验证集样本数：82


In [88]:
class DatasetTask(Dataset):
    def __init__(self, _df,transforms):       
        super(DatasetTask, self).__init__()       
        self._df=_df
        self.transforms=transforms

    def __getitem__(self, index):
        img_path = self._df.iloc[index]['gem_path']
        img = Image.open(img_path)
        if img.mode != 'RGB':
            img = img.convert('RGB') 
        img = img.resize((224, 224), Image.BILINEAR)
        img = np.array(img).astype('float32')
        img = img.transpose((2, 0, 1)) / 255
        if self.transforms is not None:
          img = self.transforms(img)
        label = self._df.iloc[index]['lbl']
        label = np.array([label], dtype="int64")
        return img, label

    def print_sample(self, index: int = 0):
        """ 打印示例 """
        print("文件名", self._df.iloc[index]['gem_path'], "\t标签值", self._df.iloc[index]['lbl'])

    def __len__(self):
        return len(self._df)

In [89]:
import paddle.vision.transforms as T

transforms = T.Compose([
                T.RandomHorizontalFlip(0.5),#水平翻转
                T.RandomRotation(15),#随机反转角度范围
                T.RandomVerticalFlip(0.15),
                T.RandomRotation(15)
])

In [90]:
train_dataset=DatasetTask(train_data,transforms)
eval_dataset=DatasetTask(eval_data,transforms)

#训练数据加载
train_loader = paddle.io.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#测试数据加载
eval_loader = paddle.io.DataLoader(eval_dataset, batch_size = 8, shuffle=False)

In [91]:
train_dataset.print_sample(200)
print(train_dataset.__len__())
eval_dataset.print_sample(0)
print(eval_dataset.__len__())
print(eval_dataset.__getitem__(10)[0].shape)
print(eval_dataset.__getitem__(10)[1].shape)

文件名 /home/aistudio/data/dataset/Quartz Beer/quartz beer_16.jpg 	标签值 19
729
文件名 /home/aistudio/data/dataset/Beryl Golden/beryl golden_29.jpg 	标签值 3
82
(3, 224, 224)
(1,)


In [92]:
Batch=0
Batchs=[]
all_train_accs=[]
def draw_train_acc(Batchs, train_accs):
    title="training accs"
    plt.title(title, fontsize=24)
    plt.xlabel("batch", fontsize=14)
    plt.ylabel("acc", fontsize=14)
    plt.plot(Batchs, train_accs, color='green', label='training accs')
    plt.legend()
    plt.grid()
    plt.show()

all_train_loss=[]
def draw_train_loss(Batchs, train_loss):
    title="training loss"
    plt.title(title, fontsize=24)
    plt.xlabel("batch", fontsize=14)
    plt.ylabel("loss", fontsize=14)
    plt.plot(Batchs, train_loss, color='red', label='training loss')
    plt.legend()
    plt.grid()
    plt.show()

# 2、定义模型

In [93]:
!pip install loguru

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip available: 22.1.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [94]:
import paddle
from paddle import nn as nn
from loguru import logger
# 基本上原样模仿facenet-pytorch中的网络
class SelfModel2(nn.Layer):
    def __init__(self,num_classes,pretrained=True):
        super(SelfModel2,self).__init__() 
        self.backbone=paddle.vision.mobilenet_v2(pretrained=pretrained,with_pool=False,num_classes=num_classes)
        state_dict=self.backbone.state_dict()
        # for key in state_dict:
        #     logger.debug(f'{key} {state_dict[key].shape}')

        del self.backbone.classifier

        self.avg=nn.AdaptiveAvgPool2D((1,1))
        self.drop=nn.Dropout(0.5)
        self.fc=nn.Linear(1280,128,bias_attr=False)
        self.last_bn=nn.BatchNorm1D(128,epsilon=0.001,momentum=0.1)
        self.classifier=nn.Linear(128,num_classes)

    def forward(self,input):       
        x=self.backbone.features(input)
        x=self.avg(x)
        x = paddle.flatten(x, 1)
        x=self.drop(x)
        #print(x.shape)
        x=self.fc(x)
        before_normalize=self.last_bn(x)
        x=nn.functional.normalize(x,p=2,axis=1)
        cls=self.classifier(before_normalize)
        return cls

In [ ]:
# x=paddle.zeros((1,3,224,224),dtype='float32')
# net=SelfModel2(25)
# out=net(x)
# print(len(out))

In [95]:
import scipy
import os

import numpy as np
import paddle
from visualdl import LogWriter
import datetime
class LossHistory():
    def __init__(self,log_dir):
        date_str=datetime.datetime.strftime(datetime.datetime.now(),'%Y%m%d_%H%M%S')
        self.log_dir=os.path.join(log_dir,date_str)
        if not os.path.exists(self.log_dir):
            os.makedirs(self.log_dir)

        self.log_wrt=LogWriter(logdir=self.log_dir)
        self.losses=[]  
        self.accs=[]
       

    def log_epoch(self,epoch,avg_acc,avg_loss,avg_eval_acc,avg_eval_loss,lr):
        self.log_wrt.add_scalar(tag='train/avg_loss',value=avg_loss,step=epoch)
        self.log_wrt.add_scalar(tag='train/avg_acc',value=avg_acc,step=epoch)
        self.log_wrt.add_scalar(tag='eval/avg_eval_loss',value=avg_eval_loss,step=epoch)
        self.log_wrt.add_scalar(tag='eval/avg_eval_acc',value=avg_eval_acc,step=epoch)
        self.log_wrt.add_scalar(tag='train/lr',value=lr,step=epoch)
        self.losses.append(avg_loss)
        self.accs.append(avg_acc)

    def log(self,epoch,batch_id,total_iter,train_loss,train_acc):
        
        #self.avg_loss_acc = np.around(self.avg_loss_acc, decimals=6)        
        self.log_wrt.add_scalar(tag='train/loss', step=total_iter, value=train_loss.numpy().item())
        self.log_wrt.add_scalar(tag='train/acc', step=total_iter, value=train_acc.numpy().item())
    
    def loss_plot(self):
        iters = range(len(self.losses))

        plt.figure()
        plt.plot(iters, self.losses, 'red', linewidth = 2, label='train loss')
        # plt.plot(iters, self.val_loss, 'coral', linewidth = 2, label='val loss')
        try:
            if len(self.losses) < 25:
                num = 5
            else:
                num = 15
            plt.plot(iters, scipy.signal.savgol_filter(self.losses, num, 3), 'green', linestyle = '--', linewidth = 2, label='smooth train loss')
            # plt.plot(iters, scipy.signal.savgol_filter(self.val_loss, num, 3), '#8B4513', linestyle = '--', linewidth = 2, label='smooth val loss')
        except:
            pass
        plt.grid(True)
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend(loc="upper right")
        plt.savefig(os.path.join(self.log_dir, "epoch_loss.png"))
        plt.cla()
        plt.close("all")

        plt.figure()
        plt.plot(iters, self.accs, 'red', linewidth = 2, label='acc')
        try:
            if len(self.losses) < 25:
                num = 5
            else:
                num = 15
            plt.plot(iters, scipy.signal.savgol_filter(self.acc, num, 3), 'green', linestyle = '--', linewidth = 2, label='smooth lfw acc')
        except:
            pass
        plt.grid(True)
        plt.xlabel('Epoch')
        plt.ylabel('Lfw Acc')
        plt.legend(loc="upper right")
        plt.savefig(os.path.join(self.log_dir, "epoch_acc.png"))
        plt.cla()
        plt.close("all")
        

# 3、训练模型

## 预热学习率

In [ ]:
# !pip install -U paddlenlp==2.4.3
#!pip install -U paddlepaddle-gpu -i https://mirror.baidu.com/pypi/simple

In [96]:
from functools import partial
import math
def get_lr_scheduler(lr_decay_type, max_lr, min_lr, total_epoch, warmup_epoch_ratio = 0.1, warmup_lr_ratio = 0.1, no_aug_epoch_ratio = 0.3, step_num = 10):
    def yolox_warm_cos_lr(max_lr, min_lr, total_epoch, warmup_total_epoch, warmup_lr_start, no_aug_epoch, epoch):

        if epoch <= warmup_total_epoch: # 预热
            # lr = (lr - warmup_lr_start) * iters / float(warmup_total_epoch) + warmup_lr_start
            # 预热过程中，学习率每代前进多少
            max_lr = (max_lr - warmup_lr_start) * pow(epoch / float(warmup_total_epoch), 2) + warmup_lr_start
        elif epoch >= total_epoch - no_aug_epoch: # 不对学习率进行计算
            max_lr = min_lr
        else: #预热和不对学习率进行计算之间的代
            max_lr = min_lr + 0.5 * (max_lr - min_lr) * (
                1.0
                + math.cos(math.pi * (epoch - warmup_total_epoch) / (total_epoch - warmup_total_epoch - no_aug_epoch))
                # (现在代-预热总代数)/(总代-预热总代-退火总代)
            )
        return max_lr

    # 非cos预热时如何计算
    def step_lr(max_lr, decay_rate, step_size, epoch):
        if step_size < 1:
            raise ValueError("step_size must above 1.")
        n       = epoch // step_size
        out_lr  = max_lr * decay_rate ** n
        return out_lr

    if lr_decay_type == "cos":
        # 预热多少代
        warmup_total_epoch  = min(max(warmup_epoch_ratio * total_epoch, 1), 3)
        # 预热开始的学习率
        warmup_lr_start     = max(warmup_lr_ratio * max_lr, 1e-6)
        no_aug_epoch         = min(max(no_aug_epoch_ratio * total_epoch, 1), 15)
        func = partial(yolox_warm_cos_lr, max_lr, min_lr, total_epoch, warmup_total_epoch, warmup_lr_start, no_aug_epoch)
    else:
        decay_rate  = (min_lr / max_lr) ** (1 / (step_num - 1))
        step_size   = total_epoch / step_num
        func = partial(step_lr, max_lr, decay_rate, step_size)

    return func

def set_optimizer_lr(optimizer, lr_scheduler_func, epoch):
    lr = lr_scheduler_func(epoch)
    logger.info(f'current lr {lr}')
    optimizer.set_lr(lr)
    return lr

In [ ]:
# 停止更新，可以
# net=SelfModel2(25,pretrained=False)
# _state_dict=net.backbone.parameters()
# for key in _state_dict:
#     key.stop_gradient=True


# _state_dict=net.backbone.state_dict()
# for key in _state_dict:
#    print(_state_dict[key].stop_gradient)

In [ ]:
# 不行，不能停止更新
# net=SelfModel2(25,pretrained=False)
# net.backbone.stop_gradient=False
# _state_dict=net.backbone.state_dict()
# for key in _state_dict:
#    print(key,_state_dict[key].stop_gradient)

In [97]:
!pip show paddlepaddle-gpu

Name: paddlepaddle-gpu
Version: 2.0.2.post101
Summary: Parallel Distributed Deep Learning
Home-page: UNKNOWN
Author: 
Author-email: Paddle-better@baidu.com
License: Apache Software License
Location: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages
Requires: astor, decorator, gast, numpy, Pillow, protobuf, requests, six
Required-by: 


In [ ]:
def fit_one(model,opt,start_epoch,end_epoch,cross_entropy,loss_history,scheduler,train_loader,eval_loader):
    for pass_num in range(start_epoch,end_epoch):
        accs=[]
        losses=[]
        eval_accs=[]
        eval_losses=[]
        # if pass_num>0:break
        model.train()
        lr=set_optimizer_lr(opt,scheduler,pass_num-start_epoch)
        for batch_id,data in enumerate(train_loader()):
            # if batch_id>0:break
            image = data[0]
            label = data[1]

            predict=model(image) #数据传入model

            loss=cross_entropy(predict,label)
            acc=paddle.metric.accuracy(predict,label)#计算精度
            accs.append(acc.numpy().item())
            losses.append(loss.numpy().item())
            loss_history.log(pass_num,batch_id,batch_id+pass_num*batch_size,loss,acc)

            loss.backward()
            opt.step()
            opt.clear_grad()   #opt.clear_grad()来重置梯度
        model.eval()
        for batch_id,data in enumerate(eval_loader()):#测试集
            image=data[0]
            label=data[1]
            predict=model(image)
            loss=cross_entropy(predict,label)
            acc=paddle.metric.accuracy(predict,label)

            eval_accs.append(acc.numpy().item())
            eval_losses.append(loss.numpy().item())


            #scheduler.step()
        avg_acc=np.mean(accs)
        avg_loss=np.mean(losses)
        avg_eval_loss=np.mean(eval_losses)
        avg_eval_acc=np.mean(eval_accs)
        #loss_writer.add_scalar(tag="train/acc",value=avg_acc,step=pass_num)
        loss_history.log_epoch(pass_num, avg_acc, avg_loss,avg_eval_acc,avg_eval_loss,lr)

In [ ]:
# 冻结网络
import datetime
# from paddlenlp.transformers import *
loss_history=LossHistory('log')
model=SelfModel2(25) #模型实例化
#model=paddle.vision.mobilenet_v1(pretrained=True,num_classes=25)

cross_entropy = paddle.nn.CrossEntropyLoss()

epochs_num=train_parameters['num_epochs'] #迭代次数
optimizer_type='adam'
max_lr = 1e-3
min_lr = max_lr * 0.01
nbs = 64
lr_limit_max = 1e-3 if optimizer_type == 'adam' else 1e-1  # 0.001
lr_limit_min = 3e-4 if optimizer_type == 'adam' else 5e-4  # 0.0003
# init_lr=e-3, min_lr=e-5
# 自适应最大学习率不能小于最小，不能大于最大 min(max(0.0015,0.0003),0.001)
max_lr_fit = min(max(batch_size / nbs * max_lr, lr_limit_min), lr_limit_max)
# 自适应最小，不小于最小的0.01，不大于最大学习率的0.01  min(max(1.5e-5,e-7),e-5)
min_lr_fit = min(max(batch_size / nbs * min_lr, lr_limit_min * 1e-2), lr_limit_max * 1e-2)
logger.info(f' {max_lr_fit} {min_lr_fit}')
#   获得学习率下降的公式

#scheduler = CosineAnnealingWithWarmupDecay(max_lr_fit,min_lr_fit,0.1*epochs_num,0.4*epochs_num,verbose=True)
scheduler =get_lr_scheduler('cos', max_lr_fit, min_lr_fit, epochs_num)
opt=paddle.optimizer.Adam(learning_rate=max_lr_fit, parameters=model.parameters())
for key in model.backbone.parameters():
    key.stop_gradient = True
fit_one(model,opt,0,epochs_num,cross_entropy,loss_history,scheduler,train_loader,eval_loader)

postfix=datetime.datetime.strftime(datetime.datetime.now(),'%Y%m%d_%H%M%S')
#obj = {'model': model.state_dict(), 'opt': opt.state_dict(), 'epoch': pass_num}
path = f'selfmodel-{postfix}.pdparams'
paddle.save(model.state_dict(), path)

#draw_train_acc(Batchs,all_train_accs)
#draw_train_loss(Batchs,all_train_loss)

/home/aistudio/.data/webide/pip/lib/python3.7/site-packages/paddle/fluid/dygraph/layers.py:1483: UserWarning: Skip loading for classifier.1.weight. classifier.1.weight receives a shape [1280, 1000], but the expected shape is [1280, 25].
  warnings.warn(("Skip loading for {}. ".format(key) + str(err)))
/home/aistudio/.data/webide/pip/lib/python3.7/site-packages/paddle/fluid/dygraph/layers.py:1483: UserWarning: Skip loading for classifier.1.bias. classifier.1.bias receives a shape [1000], but the expected shape is [25].
  warnings.warn(("Skip loading for {}. ".format(key) + str(err)))
2022-11-18 20:54:32.097 | INFO     | __main__:<module>:22 -  0.001 1e-05
2022-11-18 20:54:32.102 | INFO     | __main__:set_optimizer_lr:44 - current lr 0.0001
/home/aistudio/.data/webide/pip/lib/python3.7/site-packages/paddle/nn/layer/norm.py:654: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")
2022-11-18 20:55:30.76

In [ ]:
# 不冻结网络
import datetime

cross_entropy = paddle.nn.CrossEntropyLoss()
optimizer_type='adam'
max_lr = 1e-3
min_lr = max_lr * 0.01
nbs = 64
lr_limit_max = 1e-3 if optimizer_type == 'adam' else 1e-1  # 0.001
lr_limit_min = 3e-4 if optimizer_type == 'adam' else 5e-4  # 0.0003

max_lr_fit = min(max(batch_size / nbs * max_lr, lr_limit_min), lr_limit_max)
min_lr_fit = min(max(batch_size / nbs * min_lr, lr_limit_min * 1e-2), lr_limit_max * 1e-2)
logger.info(f' {max_lr_fit} {min_lr_fit}')
#   获得学习率下降的公式

#scheduler = CosineAnnealingWithWarmupDecay(max_lr_fit,min_lr_fit,0.1*epochs_num,0.4*epochs_num,verbose=True)
scheduler =get_lr_scheduler('cos', max_lr_fit, min_lr_fit, epochs_num)
opt=paddle.optimizer.Adam(learning_rate=max_lr_fit, parameters=model.parameters())
for key in model.backbone.parameters():
    key.stop_gradient = False

#训练数据加载
train_loader = paddle.io.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#测试数据加载
eval_loader = paddle.io.DataLoader(eval_dataset, batch_size = 8, shuffle=False)

fit_one(model,opt,epochs_num,200,cross_entropy,loss_history,scheduler,train_loader,eval_loader)

'''
for pass_num in range(epochs_num+1,200):
    accs=[]
    losses=[]  
    eval_accs=[]
    eval_losses=[]    
    # if pass_num>0:break
    model.train()
    lr=set_optimizer_lr(opt,scheduler,pass_num-epochs_num)
    for batch_id,data in enumerate(train_loader()):
        # if batch_id>0:break
        image = data[0]
        label = data[1]

        predict=model(image) #数据传入model
       
        loss=cross_entropy(predict,label)
        acc=paddle.metric.accuracy(predict,label)#计算精度
        accs.append(acc.numpy().item())
        losses.append(loss.numpy().item())      
        loss_history.log(pass_num,batch_id,batch_id+pass_num*batch_size,loss,acc) 
        
        loss.backward()       
        opt.step()
        opt.clear_grad()   #opt.clear_grad()来重置梯度
    model.eval()
    for batch_id,data in enumerate(eval_loader()):#测试集
        image=data[0]
        label=data[1]     
        predict=model(image)       
        loss=cross_entropy(predict,label)
        acc=paddle.metric.accuracy(predict,label)

        eval_accs.append(acc.numpy().item())
        eval_losses.append(loss.numpy().item())        


    #scheduler.step()    
    avg_acc=np.mean(accs)    
    avg_loss=np.mean(losses)
    avg_eval_loss=np.mean(eval_losses)
    avg_eval_acc=np.mean(eval_accs)
    #loss_writer.add_scalar(tag="train/acc",value=avg_acc,step=pass_num)
    loss_history.log_epoch(pass_num, avg_acc, avg_loss,avg_eval_acc,avg_eval_loss,lr)
'''

postfix=datetime.datetime.strftime(datetime.datetime.now(),'%Y%m%d_%H%M%S')
#obj = {'model': model.state_dict(), 'opt': opt.state_dict(), 'epoch': pass_num}
path = f'selfmodel-{postfix}-melt.pdparams'
paddle.save(model.state_dict(), path)

#draw_train_acc(Batchs,all_train_accs)
#draw_train_loss(Batchs,all_train_loss)

/home/aistudio/.data/webide/pip/lib/python3.7/site-packages/paddle/fluid/dygraph/layers.py:1483: UserWarning: Skip loading for classifier.1.weight. classifier.1.weight receives a shape [1280, 1000], but the expected shape is [1280, 25].
  warnings.warn(("Skip loading for {}. ".format(key) + str(err)))
/home/aistudio/.data/webide/pip/lib/python3.7/site-packages/paddle/fluid/dygraph/layers.py:1483: UserWarning: Skip loading for classifier.1.bias. classifier.1.bias receives a shape [1000], but the expected shape is [25].
  warnings.warn(("Skip loading for {}. ".format(key) + str(err)))
2022-11-18 20:38:21.734 | INFO     | __main__:<module>:21 -  0.001 1e-05
2022-11-18 20:38:21.739 | INFO     | __main__:set_optimizer_lr:44 - current lr 0.0001
/home/aistudio/.data/webide/pip/lib/python3.7/site-packages/paddle/nn/layer/norm.py:654: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")
2022-11-18 20:39:14.78

# 4、模型评估

In [ ]:
#模型评估
para_state_dict = paddle.load("selfmodel-99.pdparams")
model.set_state_dict(para_state_dict['model']) #加载模型参数
model.eval() #验证模式

accs = []

for batch_id,data in enumerate(eval_loader()):#测试集
    image=data[0]
    label=data[1]     
    predict=model(image)       
    acc=paddle.metric.accuracy(predict,label)
    accs.append(acc.numpy()[0])
    avg_acc = np.mean(accs)
print("当前模型在验证集上的准确率为:",avg_acc)

# 5、模型预测

In [ ]:
import os
import zipfile

def unzip_infer_data(src_path,target_path):
    '''
    解压预测数据集
    '''
    if(not os.path.isdir(target_path)):     
        z = zipfile.ZipFile(src_path, 'r')
        z.extractall(path=target_path)
        z.close()


def load_image(img_path):
    '''
    预测图片预处理
    '''
    img = Image.open(img_path) 
    if img.mode != 'RGB': 
        img = img.convert('RGB') 
    img = img.resize((224, 224), Image.BILINEAR)
    img = np.array(img).astype('float32') 
    img = img.transpose((2, 0, 1))  # HWC to CHW 
    img = img/255                # 像素值归一化 
    return img


infer_src_path = '/home/aistudio/data/data55032/archive_test.zip'
infer_dst_path = '/home/aistudio/data/archive_test'
unzip_infer_data(infer_src_path,infer_dst_path)

In [ ]:
'''
模型预测
'''
para_state_dict = paddle.load("MyDNN")
model = SelfModel2(25)
model.set_state_dict(para_state_dict) #加载模型参数
model.eval() #训练模式

#展示预测图片
infer_path='data/archive_test/alexandrite_3.jpg'
img = Image.open(infer_path)
plt.imshow(img)          #根据数组绘制图像
plt.show()               #显示图像

#对预测图片进行预处理

infer_imgs = []
infer_imgs.append(load_image(infer_path))
infer_imgs = np.array(infer_imgs)

label_dic = train_parameters['label_dict']
name_label={v:k for k,v in label_dic.items()}
print(name_label)

for i in range(len(infer_imgs)):
    data = infer_imgs[i]
    dy_x_data = np.array(data).astype('float32')
    dy_x_data=dy_x_data[np.newaxis,:, : ,:]
    img = paddle.to_tensor (dy_x_data)
    out = model(img)
    lab = np.argmax(out.numpy())  #argmax():返回最大数的索引

    print("第{}个样本,被预测为：{},真实标签为：{}".format(i+1,name_label[lab],infer_path.split('/')[-1].split("_")[0]))
        
print("结束")


In [ ]:
from pathlib import Path
infer_imgs = []
infer_labels=[]
num_labels=[]
for infer_path in Path(infer_dst_path).glob('*.jpg'):
    infer_imgs.append(load_image(infer_path))
    text_label=infer_path.stem.split('_')[0]
    infer_labels.append(text_label)
    num_labels.append(label_dic[text_label.capitalize()])

infer_imgs = np.array(infer_imgs)

label_dic = train_parameters['label_dict']


for i in range(len(infer_imgs)):
    data = infer_imgs[i]
    dy_x_data = np.array(data).astype('float32')
    dy_x_data=dy_x_data[np.newaxis,:, : ,:]
    img = paddle.to_tensor (dy_x_data)
    out = model(img)
    lab = np.argmax(out.numpy())  #argmax():返回最大数的索引

    print("第{}个样本,被预测为：{},真实标签为：{}".format(i+1,name_label[lab],infer_labels[i]))
        
print("结束")